In [5]:
import tensorflow as tf
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

In [17]:
import re

In [6]:
glove = KeyedVectors.load_word2vec_format("glove.6B.50d.txt.w2v", binary=False) # Slow

In [7]:
print(glove["hello"])
print(glove["world"])

[-0.38497   0.80092   0.064106 -0.28355  -0.026759 -0.34532  -0.64253
 -0.11729  -0.33257   0.55243  -0.087813  0.9035    0.47102   0.56657
  0.6985   -0.35229  -0.86542   0.90573   0.03576  -0.071705 -0.12327
  0.54923   0.47005   0.35572   1.2611   -0.67581  -0.94983   0.68666
  0.3871   -1.3492    0.63512   0.46416  -0.48814   0.83827  -0.9246
 -0.33722   0.53741  -1.0616   -0.081403 -0.67111   0.30923  -0.3923
 -0.55002  -0.68827   0.58049  -0.11626   0.013139 -0.57654   0.048833
  0.67204 ]
[-0.41486   0.71848  -0.3045    0.87445   0.22441  -0.56488  -0.37566
 -0.44801   0.61347  -0.11359   0.74556  -0.10598  -1.1882    0.50974
  1.3511    0.069851  0.73314   0.26773  -1.1787   -0.148     0.039853
  0.033107 -0.27406   0.25125   0.41507  -1.6188   -0.81778  -0.73892
 -0.28997   0.57277   3.4719    0.73817  -0.044495 -0.15119  -0.93503
 -0.13152  -0.28562   0.76327  -0.83332  -0.6793   -0.39099  -0.64466
  1.0044   -0.2051    0.46799   0.99314  -0.16221  -0.46022  -0.37639
 -0.6754

In [8]:
from tensorflow.keras import layers

In [25]:
def encode(text):
    x = []
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', "", text)
    words = text.split()
    for word in words:
        x.append(glove[word])
    return x

In [26]:
encode("Hello World7,.")

KeyError: "word 'helloworld' not in vocabulary"